## Config

In [1]:
# parameters
SAVE_OUTPUT = True
SIZE_PLOTS = (12,10)
YEAR = 2017

#Location of the data
INPUT_DATA_PATH = "../data/raw/"
INPUT_PROCESSED_DATA_PATH = "../data/processed/"
OUTPUT_DATA_PATH = "../data/interim/"

In [2]:
# Parameters
YEAR = 2024
SAVE_OUTPUT = True


In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geoarrow.pandas as _
import arrow
from pathlib import Path
import json
import csv

## Load data

In [4]:
df = pd.read_csv(f"{INPUT_DATA_PATH}/{YEAR}_aforament_descripcio.csv", sep=",", on_bad_lines='skip')
stations_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitud, df.Latitud))
stations_data = stations_data.set_crs(epsg=4326)

## Data management

In [5]:
print(stations_data.shape)
print(stations_data.columns)
stations_data.head()

(838, 14)
Index(['Id_aforament', 'Desc_aforament', 'Codi_tipus_aforament',
       'Desc_tipus_aforament', 'Num_carrils', 'Codi_districte', 'Codi_Barri',
       'Codi_tipus_equip_mesura', 'Desc_tipus_equip_mesura', 'Longitud',
       'Latitud', 'X_ETRS89', 'Y_ETRS89', 'geometry'],
      dtype='object')


,Id_aforament,Desc_aforament,Codi_tipus_aforament,Desc_tipus_aforament,Num_carrils,Codi_districte,Codi_Barri,Codi_tipus_equip_mesura,Desc_tipus_equip_mesura,Longitud,Latitud,X_ETRS89,Y_ETRS89,geometry
0,10001,ARAGÓ - NAVAS (Llobregat),1,Trànsit,2,10,65,1,Espira,2.190735,41.411680,432364.559,4584775.026,POINT (2.19074 41.41168)
1,10002,BAC DE RODA - BOLIVIA (Pujada),1,Trànsit,3,10,71,1,Espira,2.198983,41.411337,433053.504,4584730.520,POINT (2.19898 41.41134)
2,10005,ARAGÓ - BILBAO (Besòs),1,Trànsit,2,10,65,1,Espira,2.191886,41.412286,432461.330,4584841.413,POINT (2.19189 41.41229)
3,10007,CANTÀBRIA - GUIPÚSCOA (Baixada),1,Trànsit,2,10,72,1,Espira,2.201899,41.419286,433305.332,4585610.784,POINT (2.20190 41.41929)
4,10008,CANTÀBRIA - GUIPÚSCOA (Pujada),1,Trànsit,2,10,72,1,Espira,2.201063,41.420043,433236.312,4585695.525,POINT (2.20106 41.42004)


In [6]:
stations_data.dtypes

Id_aforament                 object
Desc_aforament               object
Codi_tipus_aforament          int64
Desc_tipus_aforament         object
Num_carrils                   int64
Codi_districte                int64
Codi_Barri                    int64
Codi_tipus_equip_mesura       int64
Desc_tipus_equip_mesura      object
Longitud                    float64
Latitud                     float64
X_ETRS89                    float64
Y_ETRS89                    float64
geometry                   geometry
dtype: object

## Select Bicycle Counters

In [7]:
stations_data["Desc_tipus_aforament"].value_counts(dropna=False)

Desc_tipus_aforament
Trànsit    395
Bicis      381
Rondes      62
Name: count, dtype: int64

In [8]:
if YEAR != 2017:
    bike_stations_data=stations_data[stations_data["Desc_tipus_aforament"]=="Bicis"].copy()
else:
    bike_stations_data=stations_data[stations_data["Desc_tipus_aforament"]=="bicis"].copy()

In [9]:
bike_stations_data_clean=bike_stations_data.drop(columns=['Longitud', 'Latitud', 'X_ETRS89', 'Y_ETRS89','Codi_tipus_aforament','Desc_tipus_aforament']).copy()
bike_stations_data_clean.head()


,Id_aforament,Desc_aforament,Num_carrils,Codi_districte,Codi_Barri,Codi_tipus_equip_mesura,Desc_tipus_equip_mesura,geometry
395,20001,DIPUTACIÓ - COMTE D'URGELL (carril BICI),1,2,8,1,Espira,POINT (2.15709 41.38300)
396,20002,DIPUTACIÓ - COMTE BORELL (carrril BICI),1,2,9,1,Espira,POINT (2.15709 41.38300)
397,20003,GRAN VIA DE LES CORTS CATALANES - MUNTANER (ca...,1,2,10,1,Espira,POINT (2.16165 41.38459)
398,20005,MUNTANER - GRAN VIA DE LES CORTS CATALANES (ca...,1,2,10,1,Espira,POINT (2.16310 41.38388)
399,20006,MUNTANER - SEPULVEDA (carril BICI sentit munta...,1,2,10,1,Espira,POINT (2.16313 41.38388)


## Save output

In [10]:
if SAVE_OUTPUT:
    bike_stations_data_clean.to_parquet(f'{OUTPUT_DATA_PATH}/stations_data{YEAR}.parquet')

In [11]:
# Define the new row data
new_row = {
    'Year': YEAR,
    'Columns': stations_data.shape[1],
    'Counters': stations_data.shape[0],
    'Bike Counters': bike_stations_data.shape[0],
    'Car Counters': stations_data.shape[0]-bike_stations_data.shape[0]
}

# Convert the new row data to a DataFrame
new_row_df = pd.DataFrame([new_row])

# Define the path to the CSV file
csv_file_path = f"{INPUT_PROCESSED_DATA_PATH}/log_stations.csv"

# Check if the CSV file exists
if Path(csv_file_path).is_file():
    # If the file exists, append the new row
    new_row_df.to_csv(csv_file_path, mode='a', header=False, index=False)
else:
    # If the file does not exist, create it with the new row
    new_row_df.to_csv(csv_file_path, mode='w', header=True, index=False)

## Watermark

In [12]:
!python -m pip install watermark --quiet

In [13]:
%load_ext watermark

In [14]:
%watermark

Last updated: 2025-03-14T15:54:48.192848+01:00

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 8.25.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 25 Model 68 Stepping 1, AuthenticAMD
CPU cores   : 16
Architecture: 64bit



In [15]:
%watermark --iversions

csv      : 1.0
geopandas: 0.13.2
pandas   : 2.0.3
json     : 2.0.9
numpy    : 1.24.4
arrow    : 1.3.0



In [16]:
!lsb_release -a

"lsb_release" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
